In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import LabelEncoder
import graphviz
import os
import re


df=pd.read_csv('./data/redwine2.csv')

In [3]:

# 1. 데이터 로드
df = pd.read_csv('./data/redwine1.csv')


# 2. 'sweet', 'acidity', 'body', 'tannin' 컬럼에서 숫자만 추출하여 변환
def extract_number(text):
    if pd.isna(text):
        return None
    match = re.search(r'\d+', str(text))
    if match:
        return int(match.group(0))
    return None

df['sweet'] = df['sweet'].apply(extract_number)
df['acidity'] = df['acidity'].apply(extract_number)
df['body'] = df['body'].apply(extract_number)
df['tannin'] = df['tannin'].apply(extract_number)


# 3. 필요한 컬럼만 선택 및 결측치 제거
if 'varieties1' not in df.columns:
    print("오류: 'varieties1' 컬럼을 찾을 수 없습니다. 데이터에 해당 컬럼이 있는지 확인해주세요.")
    exit()

df_processed = df[['sweet', 'acidity', 'body', 'tannin', 'varieties1']].dropna()


# 4. 'varieties1' 컬럼 인코딩
le = LabelEncoder()
df_processed['varieties1_encoded'] = le.fit_transform(df_processed['varieties1'])

# 5. 특성(X) 및 타겟(y) 정의
features = ['sweet', 'acidity', 'body', 'tannin']
X = df_processed[features]
y = df_processed['varieties1_encoded']
class_names = le.classes_

# 6. class_names의 특수 문자 처리
cleaned_class_names = []
for name in class_names:
    cleaned_name = name.replace('&', ' and ')
    cleaned_name = re.sub(r'<[^>]+>', '', cleaned_name)
    cleaned_name = cleaned_name.strip()
    cleaned_class_names.append(cleaned_name)

# --- !!! 이 부분이 트리의 갈래를 줄이는 핵심 변경 부분 !!! ---
# 7. 의사결정 트리 모델 훈련
# max_depth를 설정하여 트리의 최대 깊이를 제한합니다.
# 초기 값으로 3이나 4를 시도해보고, 결과에 따라 값을 조정해보세요.
# min_samples_leaf나 min_samples_split을 함께 사용하면 더 세밀한 제어가 가능합니다.
dt_clf = DecisionTreeClassifier(max_depth=5, random_state=42) # <-- max_depth 추가
# dt_clf = DecisionTreeClassifier(min_samples_leaf=10, random_state=42) # min_samples_leaf도 고려
# dt_clf = DecisionTreeClassifier(min_samples_split=20, random_state=42) # min_samples_split도 고려

dt_clf.fit(X, y)

# 8. 의사결정 트리 시각화 (요청하신 .dot 파일 형식으로)

# 8-1. 의사결정 트리를 'tree.dot' 파일로 내보내기
export_graphviz(dt_clf, out_file="tree.dot",
                feature_names=features,
                class_names=cleaned_class_names,
                filled=True, rounded=True,
                special_characters=True)

# 8-2. 'tree.dot' 파일을 읽어서 dot_graph 변수에 저장
with open("tree.dot") as f:
    dot_graph = f.read()

# 8-3. dot_graph를 사용하여 graphviz 소스 객체 생성
graph = graphviz.Source(dot_graph)

# 8-4. 그래프를 파일로 출력
graph.render("wine_varieties_decision_tree_pruned", format="png") # 파일 이름 변경

# PDF로도 출력 (이전에 요청하신 부분 유지)
graph.render("wine_varieties_decision_tree_pruned", format="pdf")
print("PDF 파일도 생성되었습니다: wine_varieties_decision_tree_pruned.pdf")

# SVG로도 출력 (이전에 요청하신 부분 유지)
graph.render("wine_varieties_decision_tree_pruned", format="svg")
print("SVG 파일도 생성되었습니다: wine_varieties_decision_tree_pruned.svg")


print("의사결정 트리가 'tree.dot' 파일로 생성되었고, 이를 이용해 'wine_varieties_decision_tree_pruned.png' 파일이 생성되었습니다.")
print(f"클래스 분류 기준은 'varieties1' 컬럼(품종)입니다. 예측된 품종: {class_names.tolist()}")
print(f"Graphviz 오류 방지를 위해 처리된 품종 이름: {cleaned_class_names}")



Error: Could not open "wine_varieties_decision_tree_pruned.pdf" for writing : Permission denied


CalledProcessError: Command '[WindowsPath('dot'), '-Kdot', '-Tpdf', '-O', 'wine_varieties_decision_tree_pruned']' returned non-zero exit status 1. [stderr: b'Error: Could not open "wine_varieties_decision_tree_pruned.pdf" for writing : Permission denied\r\n']